In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Installation de kenlm



In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir kenlm/build
%cd kenlm/build
!cmake ..
!make -j2

--2020-09-25 13:15:56--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 492369 (481K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.83K  2.95MB/s    in 0.2s    

2020-09-25 13:15:56 (2.95 MB/s) - written to stdout [492369/492369]

/content/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
--

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     |████████████████████████████████| 552kB 1.4MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp36-cp36m-linux_x86_64.whl size=2326207 sha256=7a90852e338a293a08c9ec8270a277f2a5e84d5402df45f916845d050af2dd0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0rxd0ak/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm


In [ ]:
%cd ../..

/content


##Entraînement sur différents nombre de tranches

In [66]:
import os
import io
import nltk
import time
import csv
import re
import pandas as pd
import numpy as np
from nltk.tokenize import ToktokTokenizer
from string import punctuation
import matplotlib.pyplot as plt
import kenlm
import pickle

In [67]:
TRAINING_FOLDER_PATH = "gdrive/My Drive/Colab Notebooks/training-monolingual.tokenized.shuffled"
TEST_FOLDER_PATH = "gdrive/My Drive/Colab Notebooks/heldout-monolingual.tokenized.shuffled"

In [68]:
%cd $TRAINING_FOLDER_PATH
%cd ..

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/training-monolingual.tokenized.shuffled'
/content/gdrive/My Drive/Colab Notebooks
/content/gdrive/My Drive


In [ ]:
list_of_files  = os.listdir("training-monolingual.tokenized.shuffled")

In [ ]:
length = len(list_of_files)
time_each_file = list()
size_each_file = list()

for i in range(1,6):

  # Get file names
  file_name      = list_of_files[i]
  print(file_name)
  arpa_file_name = "../../../TrainedModels5grams/" + str(i) + "tranches"
  #start = time.time()

  # Train language model
  !cat `ls training-monolingual.tokenized.shuffled/* | head -n $i` | ../../../kenlm/build/bin/lmplz -o 5 > {arpa_file_name}.arpa
  !../../../kenlm/build/bin/build_binary {arpa_file_name}.arpa {arpa_file_name}.binary 

  #end = time.time()
  #total_time = end - start
  #size_each_file.append(os.path.getsize(arpa_file_name + ".binary"))
  #time_each_file.append(total_time)
  

In [ ]:
with open("time_and_size_24-30.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(zip(size_each_file, time_each_file))

In [ ]:
!cat heldout-monolingual.tokenized.shuffled/news.en-00000-of-00100 | head -1000 > first1000sentences_test.txt

In [ ]:
with open("first1000sentences_test.txt") as f:
  first1000sentence_test = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
first1000sentence_test = [x.strip() for x in first1000sentence_test] 

In [ ]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

for models in os.listdir("TrainedLanguageModels"):
  perplexity_list = list()

  if "binary" in models:
    print(models)
    model_train = kenlm.Model('TrainedLanguageModels/' + models)

    for sentence in first10000sentence_test:
      perplexity_list.append(model_train.perplexity(sentence))

    numpy_perplexity = np.asarray(perplexity_list)
    _min = min(numpy_perplexity)
    _max = max(numpy_perplexity)
    _moy = sum(numpy_perplexity)/len(numpy_perplexity)

    perplexity_min.append(_min)
    perplexity_max.append(_max)
    perplexity_moy.append(_moy)

for models in sorted(os.listdir("../../../TrainedModels"), key=lambda x: int(x[0:2])):
  perplexity_list = list()

  if "binary" in models:
    print(models)
    model_train = kenlm.Model('../../../TrainedModels/' + models)

    for sentence in first10000sentence_test:
      perplexity_list.append(model_train.perplexity(sentence))

    numpy_perplexity = np.asarray(perplexity_list)
    _min = min(numpy_perplexity)
    _max = max(numpy_perplexity)
    _moy = sum(numpy_perplexity)/len(numpy_perplexity)

    perplexity_min.append(_min)
    perplexity_max.append(_max)
    perplexity_moy.append(_moy)

1tranches.binary
2tranches.binary
3tranches.binary
4tranches.binary
5tranches.binary
6tranches.binary
7tranches.binary
8tranches.binary
9tranches.binary
10tranches.binary
11tranches.binary
12tranches.binary
13tranches.binary
14tranches.binary
15tranches.binary
16tranches.binary
17tranches.binary
18tranches.binary
19tranches.binary
20tranches.binary
21tranches.binary
22tranches.binary
23tranches.binary
24tranches.binary
25tranches.binary
26tranches.binary
27tranches.binary
28tranches.binary
29tranches.binary
30tranches.binary


In [ ]:
with open("../../../REAL_min_max_moy_perplexity_10000_bigram.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(zip(perplexity_min, perplexity_max, perplexity_moy))

##Measure perplexity trained on 9 files 

In [ ]:
!cat heldout-monolingual.tokenized.shuffled/news.en-00000-of-00100 | head -1000 > first1000sentences_test.txt

with open("first1000sentences_test.txt") as f:
  first1000sentence_test = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
first1000sentence_test = [x.strip() for x in first1000sentence_test] 

In [ ]:
model9 = kenlm.Model('TrainedLanguageModels/9tranches.binary')
perplexity_list = list()

for sentence in first1000sentence_test:
  perplexity_list.append(model9.perplexity(sentence))

In [ ]:
numpy_perplexity = np.asarray(perplexity_list)

In [ ]:
index_min = np.argmin(numpy_perplexity)
index_max = np.argmax(numpy_perplexity)

In [ ]:
print(numpy_perplexity[index_max])
print(first1000sentence_test[index_max])

print(numpy_perplexity[index_min])
print(first1000sentence_test[index_min])

print(sum(numpy_perplexity)/len(numpy_perplexity))

21411.129542915227
Rossi broke Stoner 's 2007 lap record of 2min 02.108sec.
28.735341750242814
" I had never seen anything like it .
412.93303849347126


In [ ]:
model9.perplexity('first1000sentence_test')

#model9.score('first1000sentence_test', bos = True, eos = True))

647145.2646303398

##Measure different ngram

In [ ]:
!cat heldout-monolingual.tokenized.shuffled/news.en-00000-of-00100 | head -20000 > first20000sentences_test.txt

with open("first20000sentences_test.txt") as f:
  first20000sentences_test = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
first20000sentences_test = [x.strip() for x in first20000sentences_test] 

In [ ]:
!ls

first1000sentences_test.txt
first5000sentences_test.txt
heldout-monolingual.tokenized.shuffled
kenlm_project.ipynb
min_max_moy_perplexity_10000_secondtry.csv
min_max_moy_perplexity_2.csv
time_and_size_20-25.csv
time_and_size_24-30.csv
time_and_size.csv
TrainedLanguageModels
TrainedModelsTrigram
training-monolingual.tokenized.shuffled


In [ ]:
for i in range(2,11):

  # Get file names
  arpa_file_name = "../../../DifferentNgram/" + str(i) + "gram_10tranches"

  # Train language model
  !cat `ls training-monolingual.tokenized.shuffled/* | head -n 10` | ../../../kenlm/build/bin/lmplz -o $i > {arpa_file_name}.arpa
  !../../../kenlm/build/bin/build_binary {arpa_file_name}.arpa {arpa_file_name}.binary 
  

In [ ]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

for models in sorted(os.listdir("../../../DifferentNgram")[:-1], key=lambda x: int(x[0:2])):
  perplexity_list = list()

  if "binary" in models:
    print(models)
    model_train = kenlm.Model('../../../DifferentNgram/' + models)

    for sentence in first20000sentences_test:
      perplexity_list.append(model_train.perplexity(sentence))

    numpy_perplexity = np.asarray(perplexity_list)
    _moy = sum(numpy_perplexity)/len(numpy_perplexity)

    perplexity_moy.append(_moy)

02gram_10tranches.binary
03gram_10tranches.binary
04gram_10tranches.binary
05gram_10tranches.binary
06gram_10tranches.binary


In [ ]:
perplexity_moy

[423.6431910999242,
 332.1688324369908,
 318.55618076953016,
 316.43142967918834,
 315.9849279005555]

In [ ]:
with open("../../../moy_perplexity_20000_diffgram2-6.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(zip(perplexity_moy))

##Entraîner modèles sur texte prétraité

In [ ]:
# Get file names
arpa_file_name = "CleanedModels/" + "2gram_10tranches"

# Train language model
!kenlm/build/bin/lmplz -o 2 <clean_train.txt> {arpa_file_name}.arpa

In [128]:
!kenlm/build/bin/build_binary -s {arpa_file_name}.arpa {arpa_file_name}.binary 

Reading CleanedModels/2gram_10tranches.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [90]:
%cd "gdrive/My Drive/Colab Notebooks"

/content/gdrive/My Drive/Colab Notebooks


In [92]:
!cat heldout-monolingual.tokenized.shuffled/news.en-00000-of-00100 | head -10000 > first10000.txt

In [97]:
from nltk.tokenize import ToktokTokenizer
nltk.download('wordnet')
  
# Initialise the lemmatizer
lemmatizer = nltk.WordNetLemmatizer()

# Initialise the tokenizer
tokenizer = ToktokTokenizer()
tokenizer.AMPERCENT = re.compile('& '), '& '
tokenizer.TOKTOK_REGEXES = [(regex, sub) if sub != '&amp; ' else (re.compile('& '), '& ') for (regex, sub) in
                            ToktokTokenizer.TOKTOK_REGEXES]
toktok = tokenizer.tokenize

In [111]:
with open("first10000.txt") as f:
  first10000sentences_test = f.readlines()

In [121]:
list_of_sentence = list()

# you may also want to remove whitespace characters like `\n` at the end of each line
for x in first10000sentences_test:
  splitted = toktok(x.strip().lower())
  lemmatized = [lemmatizer.lemmatize(t) for t in splitted]
  tokens = pd.Series(lemmatized)
  tokens.loc[tokens.apply(lambda x: x.isnumeric())] = "NUMBER"
  tokens = ' '.join(word for word in tokens)
  list_of_sentence.append(tokens)

In [130]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

model = "CleanedModels/2gram_10tranches.binary"
perplexity_list = list()

model_train = kenlm.Model(model)

for sentence in list_of_sentence:
  perplexity_list.append(model_train.perplexity(sentence))

numpy_perplexity = np.asarray(perplexity_list)
_min = min(numpy_perplexity)
_max = max(numpy_perplexity)
_moy = sum(numpy_perplexity)/len(numpy_perplexity)

perplexity_moy.append(_moy)
perplexity_min.append(_min)
perplexity_max.append(_max)

In [131]:
perplexity_moy

[1400.7957349329083]

In [132]:
perplexity_min

[60.638946054265304]

In [133]:
perplexity_max

[783331.6855142096]

##Best model


In [138]:
arpa_file_name = "20tranches_4grams"
#start = time.time()

# Train language model
!cat `ls training_set/* | head -n 20` | kenlm/build/bin/lmplz -o 4 > {arpa_file_name}.arpa
!kenlm/build/bin/build_binary {arpa_file_name}.arpa {arpa_file_name}.binary 

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
tcmalloc: large alloc 2179538944 bytes == 0x55e854ee2000 @  0x7f052bfa41e7 0x55e85272f7a2 0x55e8526ca51e 0x55e8526a92eb 0x55e852695066 0x7f052a13db97 0x55e852696baa
tcmalloc: large alloc 8718139392 bytes == 0x55e8d6d74000 @  0x7f052bfa41e7 0x55e85272f7a2 0x55e85271e7ca 0x55e85271f208 0x55e8526a9308 0x55e852695066 0x7f052a13db97 0x55e852696baa
Unigram tokens 150873690 types 985666
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:11827992 2:1857192192 3:3482235392 4:5571576832
tcmalloc: large alloc 5571584000 bytes == 0x55e854de0000 @  0x7f052bfa41e7 0x55e85272f7a2 0x55e85271e7ca 0x55e85271f208 0x55e8526a98d7 0x55e852695066 0x7f052a13db97 0x55e852696baa
tcmalloc: large alloc 1857200128 bytes == 0x55e9a1aaa000 @  0x7f052bfa41e7 0x55e85272f7a2 0x55e85271e7ca 0x55e85271f208 0x55e8526a9cdd 0x55e852695066 0x7f052a13db97 0x55e852696baa
tcmalloc: large al

In [139]:
arpa_file_name = "20tranches_3grams"
#start = time.time()

# Train language model
!cat `ls training_set/* | head -n 20` | kenlm/build/bin/lmplz -o 3 > {arpa_file_name}.arpa
!kenlm/build/bin/build_binary {arpa_file_name}.arpa {arpa_file_name}.binary 

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
tcmalloc: large alloc 2514853888 bytes == 0x55eaf1e06000 @  0x7ff7962401e7 0x55eaef49f7a2 0x55eaef43a51e 0x55eaef4192eb 0x55eaef405066 0x7ff7943d9b97 0x55eaef406baa
tcmalloc: large alloc 8382824448 bytes == 0x55eb87c60000 @  0x7ff7962401e7 0x55eaef49f7a2 0x55eaef48e7ca 0x55eaef48f208 0x55eaef419308 0x55eaef405066 0x7ff7943d9b97 0x55eaef406baa
Unigram tokens 150873690 types 985666
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:11827992 2:3795132160 3:7115872768
tcmalloc: large alloc 7115874304 bytes == 0x55eaf1d04000 @  0x7ff7962401e7 0x55eaef49f7a2 0x55eaef48e7ca 0x55eaef48f208 0x55eaef4198d7 0x55eaef405066 0x7ff7943d9b97 0x55eaef406baa
tcmalloc: large alloc 3795132416 bytes == 0x55ed7bf88000 @  0x7ff7962401e7 0x55eaef49f7a2 0x55eaef48e7ca 0x55eaef48f208 0x55eaef419cdd 0x55eaef405066 0x7ff7943d9b97 0x55eaef406baa
Statistics:
1 985666 D1=0.70642

In [154]:
arpa_file_name = "20tranches_5grams"
#start = time.time()

# Train language model
!cat `ls training_set/* | head -n 20` | kenlm/build/bin/lmplz -o 5 > {arpa_file_name}.arpa
!kenlm/build/bin/build_binary {arpa_file_name}.arpa {arpa_file_name}.binary 

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
tcmalloc: large alloc 1923121152 bytes == 0x55f9749a8000 @  0x7fc916a4d1e7 0x55f972e817a2 0x55f972e1c51e 0x55f972dfb2eb 0x55f972de7066 0x7fc914be6b97 0x55f972de8baa
tcmalloc: large alloc 8974548992 bytes == 0x55f9e73b0000 @  0x7fc916a4d1e7 0x55f972e817a2 0x55f972e707ca 0x55f972e71208 0x55f972dfb308 0x55f972de7066 0x7fc914be6b97 0x55f972de8baa
Unigram tokens 150873690 types 985666
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:11827992 2:1064488192 3:1995915520 4:3193464576 5:4657136128
tcmalloc: large alloc 4657143808 bytes == 0x55f9748a6000 @  0x7fc916a4d1e7 0x55f972e817a2 0x55f972e707ca 0x55f972e71208 0x55f972dfb8d7 0x55f972de7066 0x7fc914be6b97 0x55f972de8baa
tcmalloc: large alloc 1995923456 bytes == 0x55faca48a000 @  0x7fc916a4d1e7 0x55f972e817a2 0x55f972e707ca 0x55f972e71208 0x55f972dfbcdd 0x55f972de7066 0x7fc914be6b97 0x55f972de8baa
tcmal

In [141]:
!cat heldout-monolingual.tokenized.shuffled/news.en-00000-of-00100 | head -10000 > first10000sentences_test.txt

with open("first10000sentences_test.txt") as f:
  first10000sentences_test = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
first10000sentences_test = [x.strip() for x in first10000sentences_test] 

In [142]:
len(first10000sentences_test)

10000

In [144]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

model = "20tranches_4grams.binary"
perplexity_list = list()

model_train = kenlm.Model(model)

for sentence in first10000sentences_test:
  perplexity_list.append(model_train.perplexity(sentence))

numpy_perplexity = np.asarray(perplexity_list)
_min = min(numpy_perplexity)
_max = max(numpy_perplexity)
_moy = sum(numpy_perplexity)/len(numpy_perplexity)

perplexity_moy.append(_moy)
perplexity_min.append(_min)
perplexity_max.append(_max)

In [147]:
perplexity_moy

[263.14790174499075]

In [148]:
perplexity_min

[2.6118528844732403]

In [149]:
perplexity_max

[66545.5658108565]

In [150]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

model = "20tranches_3grams.binary"
perplexity_list = list()

model_train = kenlm.Model(model)

for sentence in first10000sentences_test:
  perplexity_list.append(model_train.perplexity(sentence))

numpy_perplexity = np.asarray(perplexity_list)
_min = min(numpy_perplexity)
_max = max(numpy_perplexity)
_moy = sum(numpy_perplexity)/len(numpy_perplexity)

perplexity_moy.append(_moy)
perplexity_min.append(_min)
perplexity_max.append(_max)

In [151]:
perplexity_moy

[278.38920611713894]

In [152]:
perplexity_min

[5.1386751562586435]

In [153]:
perplexity_max

[68152.45401137498]

In [155]:
perplexity_min = list()
perplexity_max = list()
perplexity_moy = list()

model = "20tranches_5grams.binary"
perplexity_list = list()

model_train = kenlm.Model(model)

for sentence in first10000sentences_test:
  perplexity_list.append(model_train.perplexity(sentence))

numpy_perplexity = np.asarray(perplexity_list)
_min = min(numpy_perplexity)
_max = max(numpy_perplexity)
_moy = sum(numpy_perplexity)/len(numpy_perplexity)

perplexity_moy.append(_moy)
perplexity_min.append(_min)
perplexity_max.append(_max)

In [156]:
perplexity_moy

[260.4249690341212]

In [157]:
perplexity_min

[2.1219458986311728]

In [158]:
perplexity_max

[66285.63569612926]